Hw2-Spaceship titanic
簡介/陳述:這次的作業是要去kaggle挑戰一個競賽，在得知這個訊息後其實我沒有想法，於是點開了老師給的連結們，最後選擇了這個太空版的鐵達尼號來當作主題，應該可以把原因歸類為以下:
1.競賽名字有鐵達尼號
2.太空船版聽起來很酷
3.我從小有太空人的夢想

雖然聽起來很好笑，但這應該就是最初我選這個主題的動機。在閱讀這個競賽的故事背景和說明之後，簡單來說，就是跟原版鐵達尼號撞上冰山有異曲同工之妙，這艘太空船上的乘客遇上被傳送到另一個維度空間的危機!然後要我們寫程式預測他們可能失蹤的人的特徵之類的。

那他一樣有提供我們乘客的各項資料，首先一定是先進行資料清理嘛，那就讓我們開始吧!
首先先引入我們需要的套件

In [1]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#matplotlib inline
import seaborn as sns

接下來是讀取這個csv檔，然後看一下這筆訓練集的資料組成(長得跟原本的鐵達尼號範例差不多)，之後是讀取他的資料型態。

In [8]:
df=pd.read_csv('space_train.csv')
df.head()


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


發現Age、RoomService、FoodCourt、ShoppingMall、Spa、VRDeck這幾個是數值項，於是我決定先用老師上課的資料清理方法。
首先是先看缺失值的總數。

In [10]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

接著是看一下對應列的缺失值數是否超過總行數的一半。用來判斷哪些列缺失值的比例超過一半。

In [11]:
df.isnull().sum()>len(df)/2

PassengerId     False
HomePlanet      False
CryoSleep       False
Cabin           False
Destination     False
Age             False
VIP             False
RoomService     False
FoodCourt       False
ShoppingMall    False
Spa             False
VRDeck          False
Name            False
Transported     False
dtype: bool

做到這邊忽然發現，這筆資料是不是有八千多筆阿，剛剛看到的缺失值都是兩百多，離超過一半還超遠的。
那就用上課教的填補資料方式填補資料好了，反正也才200/8000=0.025的資料有缺失值而已，就先把數值項的缺項都補起來吧。
首先是年齡，但跟上課教的不一樣是這裡沒有分性別，那就直接看年齡，可是這樣就沒有transform了，於是就直接用中位數填補吧。

In [15]:
median_age = df['Age'].median()
df['Age'].fillna(median_age, inplace=True)

做到這邊，我想說反正有缺失值的項也只占整筆資料的一小部分而已，那乾脆就把有缺失值的項通通去掉，變成一份小一點點的資料不就好了。
(以防萬一還是用了一個新的變數，怕原本的資料被改調改不回來就要重新用了。)

In [17]:
new_df = df.dropna()
new_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [18]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6757 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   6757 non-null   object 
 1   HomePlanet    6757 non-null   object 
 2   CryoSleep     6757 non-null   object 
 3   Cabin         6757 non-null   object 
 4   Destination   6757 non-null   object 
 5   Age           6757 non-null   float64
 6   VIP           6757 non-null   object 
 7   RoomService   6757 non-null   float64
 8   FoodCourt     6757 non-null   float64
 9   ShoppingMall  6757 non-null   float64
 10  Spa           6757 non-null   float64
 11  VRDeck        6757 non-null   float64
 12  Name          6757 non-null   object 
 13  Transported   6757 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 745.6+ KB


好吧其實少掉的列比我想的多很多，原本八千多筆資料剩下六千七百筆了。

In [19]:
new_df.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

但雖然有了一筆乾淨的資料，但我在觀看完youtube上面一部在教學如何做這個競賽的影片時，他用了和我相同的方式清掉了所有有缺失值的列，最後跑出來也是少了一千多筆資料，影片裡的結論是不太好，我個人也是這樣覺得，所以我就按照影片裡給的建議去做，嘗試用值去填補這些空缺:
1.非數值項用最常出現的值來填補
2.數值向用平均數來填補

非數值項:

In [31]:
df['PassengerId'].fillna(df['PassengerId'].mode()[0],inplace=True)
df['HomePlanet'].fillna(df['HomePlanet'].mode()[0],inplace=True)
df['CryoSleep'].fillna(df['CryoSleep'].mode()[0],inplace=True)
df['Cabin'].fillna(df['Cabin'].mode()[0],inplace=True)
df['Destination'].fillna(df['Destination'].mode()[0],inplace=True)
df['VIP'].fillna(df['VIP'].mode()[0],inplace=True)
df['Name'].fillna(df['Name'].mode()[0],inplace=True)

數值項:

In [32]:
df['Age'].fillna(df['Age'].mean(),inplace=True)
df['RoomService'].fillna(df['RoomService'].mean(),inplace=True)
df['FoodCourt'].fillna(df['FoodCourt'].mean(),inplace=True)
df['ShoppingMall'].fillna(df['ShoppingMall'].mean(),inplace=True)
df['Spa'].fillna(df['Spa'].mean(),inplace=True)
df['VRDeck'].fillna(df['VRDeck'].mean(),inplace=True)
df['VRDeck'].fillna(df['VRDeck'].mean(),inplace=True)
df['Transported'].fillna(df['Transported'].mean(),inplace=True)


In [33]:
df.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

這樣就是一筆沒有缺失值的資料了!(其實上面填補缺失值那裏應該要用迴圈寫的，但是就要先到前面定義變數才能迴圈，加上我不太熟悉迴圈怎麼寫，所以就暫時採用土法煉鋼的方法了，如果老師看得很煩那還真不好意思...)

更:下面跑不出來，我回來把轉換不成數值的項通通去掉。

In [65]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           8693 non-null   float64
 1   RoomService   8693 non-null   float64
 2   FoodCourt     8693 non-null   float64
 3   ShoppingMall  8693 non-null   float64
 4   Spa           8693 non-null   float64
 5   VRDeck        8693 non-null   float64
 6   Transported   8693 non-null   bool   
dtypes: bool(1), float64(6)
memory usage: 416.1 KB


接下來就是資料的訓練和切割了，一切就照上課教過的進行吧。首先先建立訓練集和測試集。

In [67]:
X=df.drop('Transported',axis=1)
y=df['Transported']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=75)

最後就跟上課教過的一樣，用Logistic regression model去預測好了。

In [69]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,y_train)
predictions=lr.predict(X_test)

from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score
accuracy_score(y_test,predictions)
recall_score(y_test,predictions)
precision_score(y_test,predictions)

pd.DataFrame(confusion_matrix(y_test,predictions),columns=['Predict not Transported','Predict Transported'],index=['True not Transported','True Transported'])



,Predict not Transported,Predict Transported
True not Transported,827,447
True Transported,150,1184


中間的地方我把不是數值項的項還是通通去掉了，因為不知道為甚麼程式一直卡在一個人名然後就不能跑，後來刪掉就可以了。